In [4]:
from pymongo import MongoClient
import pprint

import pandas as pd
import matplotlib.pyplot as plt

import requests

from bs4 import BeautifulSoup

import json

## Request the webpage's raw HTML

In [5]:
url = 'https://www.basketball-reference.com/leagues/NBA_2019_games-october.html'
r = requests.get(url)

In [6]:
r.status_code

200

In [8]:
#pprint.pprint(r.text)

## Save the Raw HTML into a MongoDB

In [9]:
client = MongoClient()
db = client.basketball_reference

In [10]:
seasons = db.basketball_capstone1

In [11]:
seasons.insert_one({'link':url,'html':r.text})

## Parse the hypertext to get data with Beautiful Soup

In [12]:
soup = BeautifulSoup(r.text,'html.parser')

In [14]:
#print(soup.prettify())

In [22]:
soup.find_all('tr')[1] #get info for each game

<tr><th class="left" csk="201810160BOS" data-stat="date_game" scope="row"><a href="/boxscores/index.fcgi?month=10&amp;day=16&amp;year=2018">Tue, Oct 16, 2018</a></th><td class="right" data-stat="game_start_time">8:00p</td><td class="left" csk="PHI.201810160BOS" data-stat="visitor_team_name"><a href="/teams/PHI/2019.html">Philadelphia 76ers</a></td><td class="right" data-stat="visitor_pts">87</td><td class="left" csk="BOS.201810160BOS" data-stat="home_team_name"><a href="/teams/BOS/2019.html">Boston Celtics</a></td><td class="right" data-stat="home_pts">105</td><td class="center" data-stat="box_score_text"><a href="/boxscores/201810160BOS.html">Box Score</a></td><td class="center iz" data-stat="overtimes"></td><td class="right" data-stat="attendance">18,624</td><td class="left iz" data-stat="game_remarks"></td></tr>

In [118]:
soup.find_all('tr')[1].find('a').text[5:] 
#get date for each game - removed day (all dates have a three letter abbrev. with a space and ','
#five chars i.e. 'Tue, '

'Oct 16, 2018'

In [90]:
soup.find_all('tr')[1].find_all('td','left')[0].text #get name of visiting team

'Philadelphia 76ers'

In [122]:
soup.find_all('tr')[1].find_all('td','right')[1].text #get score of visiting team

'87'

In [92]:
soup.find_all('tr')[1].find_all('td','left')[1].text #get name of home team

'Boston Celtics'

In [98]:
soup.find_all('tr')[1].find_all('td','right')[2].text #get score of home team

'105'

## Gather information from other rows/games - games for the month

In [ ]:
date = []
visiting = []
visiting_score = []
home = []
home_score = []

for i,game in enumerate(soup.find_all('tr')):
    if game.a != None: #skips the first row of the chart which are just chart labels 
        date.append(game.find('a').text[5:]) 
        visiting.append(game.find_all('td','left')[0].text)
        visiting_score.append(game.find_all('td','right')[1].text)
        home.append(game.find_all('td','left')[1].text)
        home_score.append(game.find_all('td','right')[2].text)
print(visiting)

## Gather information from other months in the season